# San Joaquin Valley Existing Vegetation
Related links:
* For the documentation about this dataset, its source, how to download it, and the features of interest, please refer to our [Vegetation Datasets](doc/assets/vegetation.md) documentation
* For the explanations on how the vegetation mapping datasets are overlaid with township boundaries to obtain the amount of land used for each crop type in each township, please refer to our [Overlaying San Joaquin Valley Township Boundaries](doc/etl/township_overlay.md) documentation

In [1]:
# For Deepnote to be able to use the custom libraries in the parent ../lib folder
import sys
sys.path.append('..')

In [2]:
import matplotlib.pyplot as plt
from lib.vegetation import VegetationDataset

Load the data by instantiating the VegetationDataset class based on the geospatial map data for the central valley and central coast areas.

In [3]:
vegetation_df = VegetationDataset()

Pre-process the crops dataset to keep only the selected features for the final analysis.

In [4]:
vegetation_df.preprocess_map_df()

In [5]:
vegetation_df.map_df

,VEGETATION_TYPE,geometry,YEAR
0,Non Forest,"MULTIPOLYGON (((-121.01241 36.62881, -121.0124...",2019
1,Non Forest,"MULTIPOLYGON (((-120.91174 36.61289, -120.9117...",2019
2,Non Forest,"MULTIPOLYGON (((-120.32605 36.20565, -120.3260...",2019
3,Non Forest,"MULTIPOLYGON (((-121.53567 36.02781, -121.5356...",2019
4,Non Forest,"MULTIPOLYGON (((-119.69018 34.94837, -119.6901...",2019
...,...,...,...
54804,Non Forest,"MULTIPOLYGON (((-121.27291 38.95855, -121.2725...",2019
54805,Non Forest,"MULTIPOLYGON (((-120.64721 37.98345, -120.6472...",2019
54806,Non Forest,"MULTIPOLYGON (((-119.44758 37.03244, -119.4475...",2019
54807,Non Forest,"MULTIPOLYGON (((-119.44680 37.03424, -119.4468...",2019


Overlay the San Joaquin Valley township boundaries on the Vegetation dataset to cut the crops land areas with the township boundaries, thus extracting all the existing vegetation types per townships.

In [6]:
vegetation_df.overlay_township_boundries()

Display the map of the Vegetation dataset

In [ ]:
fig, ax = plt.subplots(figsize=(40,40))
vegetation_df.map_df.plot(ax=ax, column="VEGETATION_TYPE", edgecolor='black', linewidth = 1, cmap=None, legend=True)
plt.show()

In [9]:
vegetation_df.fill_missing_years()
vegetation_df.fill_townships_with_no_data(feature_to_fill="VEGETATION_TYPE")
vegetation_df.pivot_township_categorical_feature(feature_name="VEGETATION_TYPE", feature_prefix="VEGETATION")
vegetation_df.drop_features(drop_rate=0.05, unwanted_features=["VEGETATION_X", "VEGETATION_NON_FOREST"])

KeyboardInterrupt: 

In [ ]:
vegetation_df.output_df

In [ ]:
vegetation_df.output_df.to_csv("../assets/outputs/vegetation.csv")